In [4]:
import os
from transformers import pipeline
!pip install python-dotenv
!pip install openai
!pip install langchain-community
from dotenv import load_dotenv, find_dotenv
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from PIL import Image
from scipy.io.wavfile import write

load_dotenv(find_dotenv())
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
#create .env file and save  your openai key



def generate_caption(image_path: str) -> str:
    """Generate a caption for the uploaded image using Hugging Face's Transformers."""
    captioner = pipeline("image-to-text", model="Salesforce/blip-image-captioning-base")
    image = Image.open(image_path).convert("RGB")
    caption = captioner(image)[0]["generated_text"]
    return caption

def refine_caption(caption: str) -> str:
    """Refine the caption using LangChain and OpenAI."""
    prompt_template = PromptTemplate(input_variables=["caption"], template="Refine this image description: {caption}")
    llm = ChatOpenAI(temperature=0.7, openai_api_key=OPENAI_API_KEY)
    chain = LLMChain(llm=llm, prompt=prompt_template)
    refined_caption = chain.run(caption=caption)
    return refined_caption

def generate_speech_huggingface(text: str, output_file: str = "output_audio.wav"):
    """Generate speech from text using Hugging Face TTS models."""
    tts = pipeline("text-to-speech", model="espnet/fastspeech2-en-ljspeech")
    audio = tts(text)["array"]

    write(output_file, 22050, (audio * 32767).astype("int16"))
    return output_file

def main(image_path: str):
    if not os.path.exists(image_path):
        print(f"Error: The image file {image_path} does not exist.")
        return

    print("Generating caption from the image...")
    caption = generate_caption(image_path)
    print(f"Generated Caption: {caption}")

    print("Refining caption...")
    refined_caption = refine_caption(caption)
    print(f"Refined Caption: {refined_caption}")

    print("Converting refined caption to speech...")
    audio_file = "output_audio.wav"
    generate_speech_huggingface(refined_caption, audio_file)
    print(f"Speech generated and saved to {audio_file}.")

if __name__ == "__main__":
    IMAGE_PATH = "IMG_PATH" #replace IMG_PATH with your actual image path
    main(IMAGE_PATH)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 30.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 23.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 3.2 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12
Generating caption from the image...


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/990M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/506 [00:00<?, ?B/s]

...

preprocessor_config.json:   0%|          | 0.00/287 [00:00<?, ?B/s]

Device set to use cpu


Generated Caption: a large elephant in the water
Refining caption...


<ipython-input-4-57b4e3a953ac>:35: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(temperature=0.7, openai_api_key=OPENAI_API_KEY)
<ipython-input-4-57b4e3a953ac>:36: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template)
<ipython-input-4-57b4e3a953ac>:37: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  refined_caption = chain.run(caption=caption)


RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}